In [1]:
import json
import sqlite3


# Connect to the database
conn = sqlite3.connect('data.sqlite')

# get the database id
cursor = conn.execute("PRAGMA database_list")
# Get the name of the main database file
for row in cursor:
    if row[1] == "main":
        database_id = row[0]
        break

# print the database id
print(database_id)


# english question
question = "What is the sales of the product in the year 2013?"


def get_schema(conn):
    # Get the cursor
    cursor = conn.cursor()

    # Select all tables in the database
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    # Create an empty dictionary to store the schema
    schema = {}

    # Loop through the tables
    for table in tables:
        # Get the columns for the table
        cursor.execute("PRAGMA table_info({})".format(table[0]))
        columns = cursor.fetchall()

        # Create a list to store the column names
        column_names = []

        # Loop through the columns and get the column names
        for column in columns:
            column_names.append(column[1])

        # Add the column names to the schema for the table
        schema[table[0]] = column_names

    # Convert the schema to JSON
    json_schema = json.dumps(schema)

    # save the schema to a file
    with open('schema.json', 'w') as f:
        f.write(json_schema)

    #remove double quotes
    json_schema = json_schema.replace('"', '')

    print(json_schema)

    # return the schema
    return json_schema

    



# convert above resuslt into table_name1 | column_name1: (distinct_value1, distinct_value2, distinct_value3, distinct_value4, distinct_value5), olumn_name2: (distinct_value1, distinct_value2, distinct_value3, distinct_value4, distinct_value5) | table_name2 | column_name1: (distinct_value1, distinct_value2, distinct_value3, distinct_value4, distinct_value5), olumn_name2: (distinct_value1, distinct_value2, distinct_value3, distinct_value4, distinct_value5) ...


# def convert_json_to_table(json_schema):
#     schema = json.loads(json_schema)
#     table_name = list(schema.keys())
#     table_name.sort()
#     result = ''
#     for table in table_name:
#         result += table + ''
#         column_name = list(schema[table].keys())
#         column_name.sort()
#         for column in column_name:
#             result += " | "+column + ': ' + str(schema[table][column]) + ', '
#         result = result[:-2] + ' | '
#     return result[:-3]


# # function to create the input sequence
# def input_sequence(question, database_id, json_schema):
#     # print the result
#     db_schema = convert_json_to_table(json_schema)

#     # replace the [ and ] with ( and )
#     db_schema = db_schema.replace('[', '')
#     db_schema = db_schema.replace(']', '')

#     # replace { and } with "|"
#     # db_schema = db_schema.replace('{', '|')
#     # db_schema = db_schema.replace('}', '|')

#     # concatenate the question and database name
#     db_schema = question + ' | ' + str(database_id) + ' | ' + db_schema + ' |'

#     # remove single quotes
#     db_schema = db_schema.replace("'", "")

#     # replace "/" with "-"
#     db_schema = db_schema.replace("/", "-")

#     # return the result
#     return db_schema


# get json schema
json_schema = get_schema(conn)

# get the input sequence
# input_sequence = input_sequence(question, database_id, json_schema)

# # print the input sequence
# print(input_sequence)

# save the result to a file
with open('json_schema.json', 'w') as f:
    f.write(json_schema)

# Close the cursor and connection
cursor.close()
conn.close()


0
{train: [Row ID, Order ID, Order Date, Ship Date, Ship Mode, Customer ID, Customer Name, Segment, Country, City, State, Postal Code, Region, Product ID, Category, Sub-Category, Product Name, Sales]}


In [4]:
# convert above resuslt into table_name1 | column_name1: (distinct_value1, distinct_value2, distinct_value3, distinct_value4, distinct_value5), olumn_name2: (distinct_value1, distinct_value2, distinct_value3, distinct_value4, distinct_value5) | table_name2 | column_name1: (distinct_value1, distinct_value2, distinct_value3, distinct_value4, distinct_value5), olumn_name2: (distinct_value1, distinct_value2, distinct_value3, distinct_value4, distinct_value5) ...

import json
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.sqlite')

# Get the cursor
cursor = conn.cursor()

# Select all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Create an empty dictionary to store the schema
schema = {}

# Loop through the tables
for table in tables:
    # Get the columns for the table
    cursor.execute("PRAGMA table_info({})".format(table[0]))
    columns = cursor.fetchall()

    # Create a list to store the column names
    column_names = []

    # Loop through the columns
    for column in columns:
        column_name = column[1]
        column_names.append(column_name)

        # Get the maximum 10 distinct values for the column
        cursor.execute("SELECT DISTINCT '{}' FROM '{}' LIMIT 10".format(column_name, table[0]))
        distinct_values = cursor.fetchall()

        # Add the distinct values to the schema
        schema[table[0] + "." + column_name] = [val[0] for val in distinct_values]
    
    # Add the column names to the schema for the table
    schema[table[0]] = column_names

# Convert the schema to JSON
json_schema = json.dumps(schema)

# save the schema to a file
with open('schema.json', 'w') as f:
    f.write(json_schema)

print(json_schema)





{"train.Row ID": ["Row ID"], "train.Order ID": ["Order ID"], "train.Order Date": ["Order Date"], "train.Ship Date": ["Ship Date"], "train.Ship Mode": ["Ship Mode"], "train.Customer ID": ["Customer ID"], "train.Customer Name": ["Customer Name"], "train.Segment": ["Segment"], "train.Country": ["Country"], "train.City": ["City"], "train.State": ["State"], "train.Postal Code": ["Postal Code"], "train.Region": ["Region"], "train.Product ID": ["Product ID"], "train.Category": ["Category"], "train.Sub-Category": ["Sub-Category"], "train.Product Name": ["Product Name"], "train.Sales": ["Sales"], "train": ["Row ID", "Order ID", "Order Date", "Ship Date", "Ship Mode", "Customer ID", "Customer Name", "Segment", "Country", "City", "State", "Postal Code", "Region", "Product ID", "Category", "Sub-Category", "Product Name", "Sales"]}


In [ ]:
import torch.nn.functional as F
import torch
from transformers import AutoTokenizer, AutoModel
import sqlite3
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the model and tokenizer
tokenizer_decoder = AutoTokenizer.from_pretrained("tscholak/2jrayxos")
model_decoder = AutoModelForSeq2SeqLM.from_pretrained("tscholak/2jrayxos")

# save the tokenizer and model
tokenizer_decoder.save_pretrained('Model1/tokenizer')
model_decoder.save_pretrained('Model1/model')

# zip the Model1 folder
import shutil
shutil.make_archive('Model1', 'zip', 'Model1')

# check the size of the zip file in GB
import os
os.path.getsize('Model1.zip')/1e9

#split the 3 gb zip file 100 mb each 30 files
import os
import math

def split_file(file_path, chunk_size=100000000):
    file_name = os.path.basename(file_path)
    file_size = os.path.getsize(file_path)
    num_chunks = math.ceil(file_size / chunk_size)
    with open(file_path, 'rb') as f:
        for i in range(num_chunks):
            chunk = f.read(chunk_size)
            with open(str(i) + file_name, 'wb') as out_file:
                out_file.write(chunk)

split_file('/content/Model1.zip')

# Merge the 30 files into one zip file in OModel1.zip + 1Model1.zip + 2Model1.zip + 3Model1.zip + 4Model1.zip + 5Model1.zip + 6Model1.zip + 7Model1.zip + 8Model1.zip + 9Model1.zip + 10Model1.zip + 11Model1.zip + 12Model1.zip + 13Model1.zip + 14Model1.zip + 15Model1.zip + 16Model1.zip + 17Model1.zip + 18Model1.zip + 19Model1.zip + 20Model1.zip + 21Model1.zip + 22Model1.zip + 23Model1.zip + 24Model1.zip + 25Model1.zip + 26Model1.zip + 27Model1.zip + 28Model1.zip 
import os
import glob

def merge_files(file_path, output_path):
    with open(output_path, 'wb') as outfile:
        for fname in file_path:
            with open(fname, 'rb') as infile:
                outfile.write(infile.read())

file_path = glob.glob('/content/*.zip')
output_path = '/content/OModel1.zip'
merge_files(file_path, output_path)








# Load the model from the saved directory
tokenizer_decoder = AutoTokenizer.from_pretrained('Model1/tokenizer')
model_decoder = AutoModelForSeq2SeqLM.from_pretrained('Model1/model')

In [ ]:
# fatal: remote origin already exists.
# remove the remote origin
!git remote remove origin

# commit the code to github
!git add .
!git commit -m "commit"

# push the code to github forcefully lfs
!git push -f origin master

